In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [10]:
url = "https://guide.wisc.edu/undergraduate/letters-science/statistics/data-science-bs/#requirementstext"
page = requests.get(url)

In [11]:
text = BeautifulSoup(page.text, "html.parser")

In [21]:
# Get the headings for the previous tables

# First, we find all the tables and only keep the tables containing course info
tables = text.findAll("tbody")[2:4]

table_names = []
for table in tables:
    table_names.append(table.find("span", {"class": "courselistcomment areaheader"}).contents[0])

DS_required = []

for i,table_name in enumerate(table_names):
    ## Append table name
    cat = []
    cat.append({"Name":table_name})
    
    ## Next, get the required credits
    credit = tables[i].findAll("tr")
    
    flag = 0 # marks the start of the category
    categories = []
    for row in credit:
        # Search for rows containing numbers
        if row.find("td",{"class":"hourscol"}) and len(row.find("td",{"class":"hourscol"}).text) > 0:
            credit = row.find("td",{"class":"hourscol"}).text
            if credit == "3-4":
                credit = 1
            else:
                credit = round(int(credit)/3)

            # Start of a section!
            if row.find("a"):
                class_code = row.find("a").contents[0].replace(u'\xa0',' ')
                class_c = (credit,[class_code])
                categories.append(class_c)
            
        else:
            if row.find("a"):
                class_code = row.find("a").contents[0].replace(u'\xa0',' ')
                categories[-1][-1].append(class_code)
                
    need = (cat,categories)
    DS_required.append(need)


DS_required = [({'Name': 'Foundational Math Courses'},2,['MATH 221', 'MATH 217', 'MATH 275']),
               ({'Name': 'Foundational Math Courses'},1,['MATH 222', 'MATH 276']),
               ({'Name': 'Foundational Data Science Courses'},1,['STAT 240']),
               ({'Name': 'Foundational Data Science Courses'},1,['STAT 340']),
               ({'Name': 'Foundational Data Science Courses'},1,['COMP SCI 220', 'COMP SCI 300']),
               ({'Name': 'Foundational Data Science Courses'},1,['COMP SCI 320']),
               ({'Name': 'Foundational Data Science Courses'},1,['L I S 461'])]

In [20]:
## INFO FROM ELECTIVES

table = text.findAll("tbody")[4]
table_tr = table.findAll("tr")

def get_ele(x):
    for ele in x:
        return ele.get_text()
def replace_that(table):
    a = []
    for row in table:
        r = []
        credits = row.findAll("td", {"class": "hourscol" })
        credits = credits[0].get_text()
        
        catg = row.findAll("span",{"class":"courselistcomment areasubheader undefined"})
        catg = get_ele(catg)
        classtags= row.findAll("td", {"class": "codecol"})
        if len(classtags) > 0:
            classtags = get_ele(classtags).replace(u'\xa0',' ')
            classtags = classtags.replace(u'\u200b',' ')
        
        r.append(credits)
        r.append(catg)
        r.append(classtags)
        a.append(r)
    return(a[1:])

tb = replace_that(table_tr)
#tb = list(tb)

#Now, we aim to clean up tb

categories = []
for row in tb:
    ## Categories
    if len(row[0]) > 0:
        ## Categories
        if row[0] == "0-3":
            row[0] = 1
        elif row[0] == "6-9":
            row[0] = 3
        else:
            row[0] = round(int(row[0])/3)
        category = ({"Name":row[1]},row[0],[])
        categories.append(category)
    elif len(row[2]) > 0:
        categories[-1][-1].append(row[2])

DS_DATA = DS_required+categories
DS_DATA

[({'Name': 'Foundational Math Courses'},
  2,
  ['MATH 221', 'MATH 217', 'MATH 275']),
 ({'Name': 'Foundational Math Courses'}, 1, ['MATH 222', 'MATH 276']),
 ({'Name': 'Foundational Data Science Courses'}, 1, ['STAT 240']),
 ({'Name': 'Foundational Data Science Courses'}, 1, ['STAT 340']),
 ({'Name': 'Foundational Data Science Courses'},
  1,
  ['COMP SCI 220', 'COMP SCI 300']),
 ({'Name': 'Foundational Data Science Courses'}, 1, ['COMP SCI 320']),
 ({'Name': 'Foundational Data Science Courses'}, 1, ['L I S 461']),
 ({'Name': 'Machine Learning'},
  1,
  ['COMP SCI/ E C E/ M E  532',
   'COMP SCI/ E C E/ M E  539',
   'COMP SCI 540',
   'MATH 535',
   'STAT 451',
   'STAT 453']),
 ({'Name': 'Advanced Computing'},
  1,
  ['COMP SCI 400',
   'COMP SCI 412',
   'COMP SCI/ STAT  471',
   'COMP SCI/ MATH  513',
   'COMP SCI/ MATH  514',
   'COMP SCI/ E C E/ I SY E  524',
   'COMP SCI 564',
   'GEOG 573',
   'GEOG 574']),
 ({'Name': 'Statistical Modeling'},
  1,
  ['ECON 400',
   'ECON 410',